In [125]:
# import pandas and read in isobus log from a field
import pandas as pd

# we can give column definitions as we load in the file
_c = pd.read_csv('./data/7130-f-14.log', delimiter=' ', names=['ts', 'pgn', 'src', 'dest', 'pr', 'payload']);
_c.head()

,ts,pgn,src,dest,pr,payload
0,1.531349e+09,61444,0,255,3,6eff96382f27ff96
1,1.531349e+09,65341,0,255,3,ed2ef45022d05740
2,1.531349e+09,65466,49,255,6,ff0330fc3ffff00f
3,1.531349e+09,2048,61,1,6,d2431041fa1fffc3
4,1.531349e+09,65322,164,255,6,10402d00da040000


In [126]:
# we don't really need all the columns; to save time and space, we will only need the three columns
c = pd.read_csv('./data/7130-f-14.log', delimiter=' ', usecols=[0,1,5], names=['ts', 'pgn', 'payload']);
c.head()

,ts,pgn,payload
0,1.531349e+09,61444,6eff96382f27ff96
1,1.531349e+09,65341,ed2ef45022d05740
2,1.531349e+09,65466,ff0330fc3ffff00f
3,1.531349e+09,2048,d2431041fa1fffc3
4,1.531349e+09,65322,10402d00da040000


In [127]:
# we want to get only the engine load messsages
# so we will filter the `pgn` column by pgn number 61443
idx = c.index[c['pgn'] == 61443]
idx

Int64Index([     27,      57,     106,     140,     175,     206,     253,
                282,     320,     353,
            ...
            7217750, 7217780, 7217817, 7217848, 7217893, 7217928, 7217962,
            7217993, 7218038, 7218068],
           dtype='int64', length=198380)

In [128]:
# now we will only take out the rows with indices we found
c_ = c.loc[idx]
c_.head()

,ts,pgn,payload
27,1.531349e+09,61443,ffff40ffffffffff
57,1.531349e+09,61443,ffff41ffffffffff
106,1.531349e+09,61443,ffff40ffffffffff
140,1.531349e+09,61443,ffff41ffffffffff
175,1.531349e+09,61443,ffff41ffffffffff


In [129]:
# remove extra index column
c_ = c_.reset_index().drop(columns=['index'])

In [130]:
# now we want to create another column that contains only the 3rd byte which encodes the engine load value
c_['engine_load_hex']=c_.payload.str.slice(start=4, stop=6)
c_.head()

,ts,pgn,payload,engine_load_hex
0,1.531349e+09,61443,ffff40ffffffffff,40
1,1.531349e+09,61443,ffff41ffffffffff,41
2,1.531349e+09,61443,ffff40ffffffffff,40
3,1.531349e+09,61443,ffff41ffffffffff,41
4,1.531349e+09,61443,ffff41ffffffffff,41


In [131]:
# then we convert the hex value into decimal values and put them into a new column
c_['engine_load_percent'] = c_.engine_load_hex.apply(lambda x: int(x, 16))
c_.head()

,ts,pgn,payload,engine_load_hex,engine_load_percent
0,1.531349e+09,61443,ffff40ffffffffff,40,64
1,1.531349e+09,61443,ffff41ffffffffff,41,65
2,1.531349e+09,61443,ffff40ffffffffff,40,64
3,1.531349e+09,61443,ffff41ffffffffff,41,65
4,1.531349e+09,61443,ffff41ffffffffff,41,65


In [132]:
# the gps csv data only have gps timestamps in seconds
# we need the same thing here; that's why we round isobus timestamps to the nearest integers
import numpy as np

c_['ts_round'] = np.rint(c_.ts)
print(c_.ts_round.iat[0], c_.ts.iat[0])
c_.head()

1531349398.0 1531349398.030441


,ts,pgn,payload,engine_load_hex,engine_load_percent,ts_round
0,1.531349e+09,61443,ffff40ffffffffff,40,64,1.531349e+09
1,1.531349e+09,61443,ffff41ffffffffff,41,65,1.531349e+09
2,1.531349e+09,61443,ffff40ffffffffff,40,64,1.531349e+09
3,1.531349e+09,61443,ffff41ffffffffff,41,65,1.531349e+09
4,1.531349e+09,61443,ffff41ffffffffff,41,65,1.531349e+09


In [133]:
# since the rounded timestamps have duplicates, we need to downsample it
# the first step is to compute the engine load average for the same groups of timestamps
ts_groups = c_.groupby('ts_round')
engine_load_percent_mean = ts_groups.apply(lambda df: df['engine_load_percent'].mean())
engine_load_percent_mean

ts_round
1.531349e+09    64.300000
1.531349e+09    65.100000
1.531349e+09    63.450000
1.531349e+09    63.950000
1.531349e+09    64.350000
                  ...    
1.531360e+09    44.800000
1.531360e+09    44.450000
1.531360e+09    44.100000
1.531360e+09    41.600000
1.531360e+09    46.636364
Length: 9915, dtype: float64

In [134]:
# sanity check: do we have matching length arrays?
len(c_.ts_round.unique())

9915

In [135]:
# now we can create another data frame that contains the downsampled version of timestamp/engine load pair
d = {'gpsTimeSec': c_.ts_round.unique(), 'engine_load': engine_load_percent_mean.values}
e = pd.DataFrame(data=d)
e

,gpsTimeSec,engine_load
0,1.531349e+09,64.300000
1,1.531349e+09,65.100000
2,1.531349e+09,63.450000
3,1.531349e+09,63.950000
4,1.531349e+09,64.350000
...,...,...
9910,1.531360e+09,44.800000
9911,1.531360e+09,44.450000
9912,1.531360e+09,44.100000
9913,1.531360e+09,41.600000


In [137]:
# now we can read in our gps csv
g = pd.read_csv('./data/gps/7130/7130-f-14.csv');
g.head()

,gpsTime,lat,lon,altitude,x,y,speed
0,1531349391000,40.712881,-102.113422,1100.099976,743831.579576,4.510893e+06,1.75
1,1531349392000,40.712868,-102.113421,1099.900024,743831.738411,4.510892e+06,1.75
2,1531349393000,40.712855,-102.113420,1100.199951,743831.907544,4.510890e+06,1.75
3,1531349394000,40.712843,-102.113418,1100.599976,743832.062660,4.510889e+06,1.75
4,1531349395000,40.712832,-102.113417,1100.500000,743832.239943,4.510888e+06,1.75


In [138]:
# note that the `gpsTime` has a weird notation, we need to preprocess that
g['gpsTimeSec'] = g.gpsTime / 1000
g.head()

,gpsTime,lat,lon,altitude,x,y,speed,gpsTimeSec
0,1531349391000,40.712881,-102.113422,1100.099976,743831.579576,4.510893e+06,1.75,1.531349e+09
1,1531349392000,40.712868,-102.113421,1099.900024,743831.738411,4.510892e+06,1.75,1.531349e+09
2,1531349393000,40.712855,-102.113420,1100.199951,743831.907544,4.510890e+06,1.75,1.531349e+09
3,1531349394000,40.712843,-102.113418,1100.599976,743832.062660,4.510889e+06,1.75,1.531349e+09
4,1531349395000,40.712832,-102.113417,1100.500000,743832.239943,4.510888e+06,1.75,1.531349e+09


In [139]:
# merge two data frames by `gpsTimeSec`
eg = pd.merge(e, g, on='gpsTimeSec', how='inner')
eg.head()

,gpsTimeSec,engine_load,gpsTime,lat,lon,altitude,x,y,speed
0,1.531349e+09,64.30,1531349398000,40.712798,-102.113417,1099.400024,743832.351059,4.510884e+06,1.5
1,1.531349e+09,65.10,1531349399000,40.712788,-102.113418,1099.000000,743832.303156,4.510883e+06,1.5
2,1.531349e+09,63.45,1531349400000,40.712777,-102.113421,1098.500000,743832.084928,4.510882e+06,1.5
3,1.531349e+09,63.95,1531349401000,40.712767,-102.113426,1097.900024,743831.694614,4.510880e+06,1.5
4,1.531349e+09,64.35,1531349402000,40.712758,-102.113432,1097.300049,743831.205047,4.510879e+06,1.5


In [140]:
mapbox_token = open('_mapbox_token').readline().rstrip() # read in the mapbox token from file

import plotly.graph_objs as go
from plotly.offline import iplot


# define the color scale
scl = [0,"rgb(150,0,90)"],[0.125,"rgb(0, 0, 200)"],[0.25,"rgb(0, 25, 255)"],
[0.375,"rgb(0, 152, 255)"],[0.5,"rgb(44, 255, 150)"],[0.625,"rgb(151, 255, 0)"],
[0.75,"rgb(255, 234, 0)"],[0.875,"rgb(255, 111, 0)"],[1,"rgb(255, 0, 0)"]

data = []

data.append(
    go.Scattermapbox(                                                            
        lat=eg.lat,                                                              
        lon=eg.lon,                                                              
        mode='markers',                                                          
        marker=go.scattermapbox.Marker(                                          
            size=5,                                                              
            color=eg.engine_load,
            colorscale=scl,                                                      
            cmin=0,                                                              
            cmax=100,                                                              
            opacity=0.7,                                                         
            colorbar=dict(                                                       
                thickness=20,                                                    
                titleside="right",                                               
                outlinecolor="rgba(68, 68, 68, 0)",                              
                ticks="outside",                                                 
                ticklen=3,                                                       
                showticksuffix="last",                                           
                ticksuffix = " (%)",                                           
                dtick = 10
            )                                                                    
        )                                                                        
    )                                    
)

layout = go.Layout(                                                              
    title='Engine Load Map',                                     
    autosize=True,                                                               
    hovermode='closest',                                                         
    showlegend=False,                                                            
    mapbox=go.layout.Mapbox(                                                     
        accesstoken=mapbox_token,
        center=go.layout.mapbox.Center( # you can give a center coordinate of the figure
            lat=40.774773,                                                       
            lon=-102.284607,                                                     
        ), 
        bearing=0,                                                                                                                                 
        pitch=0,                                                                 
        zoom=9,                                                                 
        style='dark'                                                             
    )                                                                            
)           

fig = go.Figure(data=data, layout=layout)
iplot(fig)